In [ ]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.contingency_tables as ct

In [ ]:
benchmark_data_path = "../resources/BenchmarkDatasetClean/"
benchmark_dataset = os.listdir(benchmark_data_path)
benchmark_save_path = "../resources/BenchmarkResults_29_9_2023_Macro_Tests_all_noise_redo_redo"
benchmark_save_path2 = "../resources/BenchmarkResults_23_9_2023_Macro_Tests_HTML_original"
benchmark_save_path1 = "../resources/BenchmarkResults_23_9_2023_Macro_Tests_new_format_added_test_redo/"

benchmark_dataset = [data for data in benchmark_dataset if data.endswith(
    ".csv") and "breast" not in data]
dfs = []
for bd in benchmark_dataset:

    pass_metrics = []
    print(bd)
    df1 = pd.read_csv(os.path.join(benchmark_save_path1, bd,
                      "Macro_"+bd+"_output_revamped.csv"))
    df2 = pd.read_csv(os.path.join(benchmark_save_path2, bd,
                      "Macro_"+bd+"_output_revamped.csv"))
    df3 = pd.read_csv(os.path.join(benchmark_save_path, bd,
                      "Macro_"+bd+"_output_revamped.csv"))
    df2["TableManipulation"] = "OriginalData"
    df1["TableManipulation"] = "OriginalData"
    df2["precision_per_cell_correctness_top1"] = df2["precision_per_cell_correctness_top1"]*0.01
    df2["recall_per_cell_correctness_top1"] = df2["recall_per_cell_correctness_top1"]*0.01
    df3["precision_per_cell_correctness_top1"] = df3["precision_per_cell_correctness_top1"]*0.01
    df3["recall_per_cell_correctness_top1"] = df3["recall_per_cell_correctness_top1"]*0.01
    df = pd.concat([df1, df2, df3], ignore_index=True)
    df["Dataset-name"] = bd
    df["F1Score_per_cell_correctness_top1"] = df.apply(lambda x: np.round(2*(x.recall_per_cell_correctness_top1*x.precision_per_cell_correctness_top1)/(
        x.recall_per_cell_correctness_top1+x.precision_per_cell_correctness_top1)) if (x.recall_per_cell_correctness_top1*x.precision_per_cell_correctness_top1) != 0 else 0, axis=1)
    dfs.append(df)
all_data = pd.concat(dfs, ignore_index=True)

attributes = {c: list(df[c].unique()) for c in df.columns if c in [
    "temperature", "tableFormat", "table_ops", "TestCase"]}

#### RQ3: 

# Average percell, @pass 1 formats- noise- vs test

In [ ]:
test_cases = ['TableColumnReorderTests',
              'TableReconstructionTests1',
              'TableTransposeTests']

In [ ]:
all_data_test_vs_format_on_all_noise_ops = all_data[(
    all_data["temperature"] == 0.0)] 
print(all_data_test_vs_format_on_all_noise_ops.shape)
test_vs_format_on_all_noise_ops = all_data_test_vs_format_on_all_noise_ops.pivot_table(
    index=["tableFormat", "TableManipulation"],
    columns='TestCase',
    values=["F1Score_per_cell_correctness_top1", 'recall_per_cell_correctness_top1',
            "precision_per_cell_correctness_top1"],  
    aggfunc=["mean"]  # Custom aggregation function to append values to a list
)
test_vs_format_on_all_noise_ops = test_vs_format_on_all_noise_ops*100
test_vs_format_on_all_noise_ops = np.round(
    test_vs_format_on_all_noise_ops, decimals=2)
print(test_vs_format_on_all_noise_ops.columns)
noises = [
    'OriginalData',
    'ShuffleRows',
    'ShuffleColumns',
    'ShuffleColumnNames',
    'SequentialColumnNames',
    'ArbitraryColumnNames',
    'TransposeTable',
    'ColumnCluster',
    'SerializeTable']
table_formats = all_data["tableFormat"].value_counts().index
indi = [(formatType, noise)
        for formatType in table_formats for noise in noises]
cols = [("mean", f"{metric}_per_cell_correctness_top1", T)
        for T in test_cases for metric in ["precision", "recall", "F1Score"]]
test_vs_format_on_all_noise_ops = test_vs_format_on_all_noise_ops[cols]
test_vs_format_on_all_noise_ops = test_vs_format_on_all_noise_ops.reindex(indi)

In [ ]:
test_vs_format_on_all_noise_ops

In [ ]:
test_vs_format_on_all_noise_ops.tail(30)

### diff with p_val 

In [ ]:
test_vs_format_on_all_noise_ops_list = all_data_test_vs_format_on_all_noise_ops.pivot_table(
    index=["tableFormat", "TableManipulation"],
    columns='TestCase',
    values=["F1Score_per_cell_correctness_top1", 'recall_per_cell_correctness_top1',
            "precision_per_cell_correctness_top1"],  
    # Custom aggregation function to append values to a list
    aggfunc=lambda x: list(x)
)

cols_agg = [(f"{metric}_per_cell_correctness_top1", T)
            for T in test_cases for metric in ["precision", "recall", "F1Score"]]

test_vs_format_on_all_noise_ops_list = test_vs_format_on_all_noise_ops_list[cols_agg]
test_vs_format_on_all_noise_ops_list = test_vs_format_on_all_noise_ops_list.reindex(
    indi)

In [ ]:
test_vs_format_on_all_noise_ops_list

In [ ]:
import scipy
test_vs_format_on_all_noise_ops_diff_with_pval = test_vs_format_on_all_noise_ops.copy(
    deep=True)
test_vs_format_on_all_noise_ops_pval = test_vs_format_on_all_noise_ops.copy(
    deep=True)
for i in test_vs_format_on_all_noise_ops_diff_with_pval.index:
    for j in test_vs_format_on_all_noise_ops_diff_with_pval.columns:
        ix = (i[0], "OriginalData")
        jj = (j[1], j[2])
        original_values = test_vs_format_on_all_noise_ops_list.loc[ix, jj]
        noise_induced_values = test_vs_format_on_all_noise_ops_list.loc[i, jj]

        if i[1] != "OriginalData":
            min_val = min(len(original_values), len(noise_induced_values))

            p_value = scipy.stats.ttest_rel(
                original_values[:min_val], noise_induced_values[:min_val])

            benoffi_corrected_p_val_threshold = 0.01/8
            subtract_from = test_vs_format_on_all_noise_ops.loc[ix, j]
            difference = test_vs_format_on_all_noise_ops_diff_with_pval.loc[i,
                                                                            j]-subtract_from
            if "-" in str(difference):
                str_diff = "{:.2f}".format(difference)
            else:
                str_diff = "+"+"{:.2f}".format(difference)
            if p_value.pvalue < benoffi_corrected_p_val_threshold:

                str_diff += "**"
            test_vs_format_on_all_noise_ops_diff_with_pval.loc[i, j] = str_diff
            test_vs_format_on_all_noise_ops_pval.loc[i, j] = p_value.pvalue

In [ ]:
test_vs_format_on_all_noise_ops_diff_with_pval

### only diff Macro

In [ ]:
test_vs_format_on_all_noise_ops_diff = test_vs_format_on_all_noise_ops.copy(
    deep=True)

In [ ]:
for i in test_vs_format_on_all_noise_ops_diff.index:
    for j in test_vs_format_on_all_noise_ops_diff.columns:
        ix = (i[0], "OriginalData")
        if i[1] != "OriginalData":
            subtract_from = test_vs_format_on_all_noise_ops.loc[ix, j]
            difference = subtract_from - \
                test_vs_format_on_all_noise_ops_diff.loc[i, j]
            if "-" in str(difference):
                str_diff = "{:.2f}".format(difference)
            else:
                str_diff = "+"+"{:.2f}".format(difference)

            test_vs_format_on_all_noise_ops_diff.loc[i, j] = str_diff

In [ ]:
test_vs_format_on_all_noise_ops_diff